## Overview
Redis transactions let us execute multiple statements in a single step. But is it ACID compliant?
- **Atomicity:** Redis doesn't provide the same type of atomicity gurantee as traditional RDBMS. Explained later.
- **Consistency:** Redis ensures internal data consistency by only allowing valid operations on its native data structures.
- **Isolation:** Redis is single threaded internally. This naturally provides high level of transaction isolation. This is similar to serializable isolation level. Other isolation level are not available.
- **Durability:** As an in-memory database, Redis doesn't provide durability out of the box. But there are varying level of persistence options available.

## Transaction
A Redis transaction is created using the `MULTI` command. Subsequent commands are queued for execution and then executed in one step when `EXEC` command is run:

```bash
$ MULTI
OK
$ SET counter 1
QUEUED
$ LPUSH cities:europe Paris Berlin London
QUEUED
$ EXEC
OK
(integer) 3
```

An error can be encountered during different phases:
- Syntax error or out of memory error before `EXEC` is run. In this case no command would be run.
```bash
$ MULTI
OK
$ SET counter 1
QUEUED
$ DSU xyz
(error) ERR unknown command 'DSU', with args beginning with: 'xyz'
$ EXEC
(error) EXECABORT Transaction discarded because of previous errors.
```
- Error happening after `EXEC` is run. In this case, some commands would get executed and others fail. There is no rollback.
```bash
$ MULTI
OK
$ SET counter 1
QUEUED
$ LPOP counter
QUEUED
$ EXEC
1) OK
2) (error) WRONGTYPE Operation against a key holding the wrong kind of value
```

To abort a transaction midway while accumulating commands, use the `DISCARD` command.
```bash
$ MULTI
OK
$ SET delayed true
QUEUED
$ DISCARD
OK
$ GET delayed
(nil)
```